<a href="https://colab.research.google.com/github/Esaipriya/AISECT-ML/blob/main/KNN_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("/content/Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
param_grid={'n_neighbors':np.arange(1,50),
            'weights':['uniform'],
            'algorithm':['auto','ball_tree', 'kd_tree', 'brute'],
            'leaf_size':[30],
            'metric':['minkowski'],
            'metric_params':[None]
            }
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X,y)

Fitting 5 folds for each of 196 candidates, totalling 980 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [30], 'metric': ['minkowski'],
                         'metric_params': [None],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'weights': ['uniform']},
             scoring='f1_weighted', verbose=3)

In [11]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [12]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [13]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_neighbors': np.int64(10), 'weights': 'uniform'}
The f1_macro: 0.9077085773874417
The confusion Matrix:
 [[237  20]
 [ 17 126]]
The report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.93       257
           1       0.86      0.88      0.87       143

    accuracy                           0.91       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.91      0.91      0.91       400



In [14]:
# Save Best Model
import pickle
filename = "KNN_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [15]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_metric,param_metric_params,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002646,0.000183,0.007093,0.000557,auto,30,minkowski,None,1,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.840275,0.911250,0.869545,0.791113,0.803125,0.843062,0.043937,83
1,0.002281,0.000474,0.006379,0.000137,auto,30,minkowski,None,2,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.825411,0.869048,0.841071,0.789506,0.873970,0.839801,0.030858,94
2,0.002196,0.000316,0.006284,0.000055,auto,30,minkowski,None,3,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.911250,0.912121,0.886039,0.803125,0.816021,0.865711,0.046962,57
3,0.001976,0.000083,0.006300,0.000103,auto,30,minkowski,None,4,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.884566,0.910218,0.897009,0.815108,0.875833,0.876547,0.032839,45
4,0.002210,0.000268,0.006524,0.000141,auto,30,minkowski,None,5,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.898006,0.937229,0.886039,0.828159,0.864838,0.882854,0.036090,41


In [18]:
import numpy as np
import pickle
import pandas as pd  # Import pandas

# Load the saved model from the specified file
loaded_model = pickle.load(open('/content/KNN_CV_best_model.sav', 'rb'))

# Get user input for age, estimated salary, and gender
age = int(input("Enter Age: "))
estimated_salary = int(input("Enter Estimated Salary: "))
gender = input("Enter Gender (Male/Female): ").lower()

# Convert gender input to numerical format (1 for Male, 0 for Female)
gender_male = 1 if gender == 'male' else 0

# Prepare the user input data as a pandas DataFrame with the correct feature names
# This matches the format the scaler was fitted on.
user_input_df = pd.DataFrame([[age, estimated_salary, gender_male]], columns=['Age', 'EstimatedSalary', 'Gender_Male'])


# Scale the user input using the same scaler 'sc' that was fitted on the training data (X)
# Note: It's crucial to use the same scaler fitted on the training data to scale new data
scaled_input = sc.transform(user_input_df)

# Make a prediction using the loaded model and the scaled user input
prediction = loaded_model.predict(scaled_input)

# Output the prediction result
if prediction[0] == 1:
    print("Purchased")
else:
    print("Not Purchased")

Enter Age: 46
Enter Estimated Salary: 67000
Enter Gender (Male/Female): female
Not Purchased
